In [2]:
import numpy as np
import sys
import cv2
import pandas as pd
from tensorflow.keras.models import load_model

In [17]:
## caffe
model = '../ssd_faceDetector/res10_300x300_ssd_iter_140000_fp16.caffemodel'
config = '../ssd_faceDetector/deploy.prototxt.txt'

# ## tensorflow
# model = './ssd_faceDetector/opencv_face_detector_uint8.pb'
# config = './ssd_faceDetector/opencv_face_detector.pbtxt.txt'

face_net = cv2.dnn.readNet(model, config)
if face_net.empty():
    print('Model read failed')
    sys.exit()
    
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Camera open failed')
    sys.exit()
    
model = load_model('reco_family_1.hdf5')
    
while True:
    ret, frame = cap.read()
    if not ret:
        print('frame with failed')
        break

    ## 입력 이미지 정제
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 117, 123), swapRB=False)
    face_net.setInput(blob)
    ## 결과
    outs = face_net.forward()

    detect = outs[0, 0, :, :]
    h, w = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        if confidence> 0.5:
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)
            text = f'{confidence * 100:.2f}%'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, text, (x1, y1-3), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
            
            prd_img = frame[y1:y2, x1:x2, :].copy()
            ret = model.predict(prd_img)
            print(ret)
            print(ret.shape)

    cv2.imshow('image', frame)
    key = cv2.waitKey(20)
    if key==27 or key==ord('q'):
        break
        
cv2.destroyAllWindows()

ImportError: Filepath looks like a hdf5 file but h5py is not available. filepath=reco_family_1.hdf5

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'int'